# Set-up workspace

Read in Python packages and mount Google Drive to connect with files. 

In [2]:
import os

In [3]:
from google.colab import drive #import drive from google colab
ROOT = "/content/drive" # default location for the drive
drive.mount(ROOT)

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/My Drive/EEOB590A/')

Install rpy2 and R packages to access R libraries and functions. 

In [5]:
!apt-get install r-base
!pip install -q rpy2
packnames = ("tidyverse", "lme4", "lmerTest", "emmeans", 
             "ggResidpanel", "data.table", "stringr")
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector
utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages(StrVector(packnames))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
r-base is already the newest version (4.1.2-1.1804.0).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


R[write to console]: Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘later’, ‘DEoptimR’, ‘htmlwidgets’, ‘lazyeval’, ‘crosstalk’, ‘promises’, ‘robustbase’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘numDeriv’, ‘estimability’, ‘mvtnorm’, ‘xtable’, ‘cowplot’, ‘plotly’, ‘qqplotr’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/later_1.3.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 63785 bytes (62 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fbfdd21d0f0> [RTYPES.NILSXP]

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library("tidyverse"); theme_set(theme_bw())
library("lme4")
library("lmerTest")
library("emmeans")
library("ggResidpanel")
library("data.table")
library("stringr")

**Data Wrangling**

# Data Wrangling

## Flume dataset

Loop through /data/flume directory and merge files into new dataframe. Rename columns to remove some of the poor naming structure and filter out 'NaN' values in response variable column.

In [ ]:
 %%R
flume <- list.files(path = "./data/raw/flume", pattern='.csv', full.names = TRUE) %>% 
  map_dfr(read_csv) %>%
  subset(SiteID != 'MAR' & SiteID != 'SPL') %>%
  #setNames(., sub("\\.csv$", "", basename(.))) %>% # to add filename as a column
  #map_dfr(read_csv, .id = "source") %>%
  rename(rainday_in = 'rainday (in)', 
         flowday_in = 'flowday (in)',
         TSS_lbs_ac = 'TSS lbs/ac') %>%
         filter(!is.na(TSS_lbs_ac)) %>%
         mutate(flow_mm = flowday_in * 25.4,
         rain_mm = rainday_in * 25.4,
         runoff_mm = flow_mm / rain_mm,
         TSS_ppm = TSS_lbs_ac * 1.12) %>%
         select(SiteID, Treatment, date, flow_mm, rain_mm, TSS_ppm)

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
# For CSV files 
flumeNA <- list.files(path = "./data/raw/flume", pattern='.csv', full.names = TRUE) %>% 
  map_dfr(read_csv) %>%
  subset(SiteID != 'MAR' & SiteID != 'SPL') %>%
  #setNames(., sub("\\.csv$", "", basename(.))) %>% # to add filename as a column
  #map_dfr(read_csv, .id = "source") %>%
  rename(rainday_in = 'rainday (in)', 
         flowday_in = 'flowday (in)',
         TSS_lbs_ac = 'TSS lbs/ac') %>%
         mutate(flow_mm = flowday_in * 25.4,
         rain_mm = rainday_in * 25.4,
         runoff_mm = flow_mm / rain_mm,
         TSS_ppm = TSS_lbs_ac * 1.12) %>%
         select(SiteID, Treatment, date, flow_mm, rain_mm, TSS_ppm)

UsageError: Cell magic `%%R` not found.


Change 'date' variable into date data type. Extract year from date and create new column with only year value.

In [ ]:
%%R
flume$date <- as.Date(flume$date, "%m/%d/%Y")
flume$Year <- as.numeric(format(flume$date, "%Y"))

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
flumeNA$date <- as.Date(flumeNA$date, "%m/%d/%Y")
flumeNA$Year <- as.numeric(format(flumeNA$date, "%Y"))

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
write.csv(flumeNA,"./data/tidy/flume.csv", row.names=F)

UsageError: Cell magic `%%R` not found.


Subselect from flume annual measurements to create a file with only TSS load observations summed during the observational periods associated with the soil pad dataset.

In [ ]:
%%R
times <- read_csv("./data/raw/site_year_time.csv", )

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R 
add_dates <- function(d) {
  data.frame(date = seq(as.Date(d$StartDate, format="%m/%d/%Y"),
                         as.Date(d$EndDate,   format="%m/%d/%Y"),
                         by = "day"))
}

In [ ]:
%%R
new_times <- times %>%
  group_by(SiteID, Year, Time) %>%
  do(add_dates(.)) %>%
  ungroup()

In [ ]:
%%R
index <- c('ARM', 'EIA', 'MCN', 'RHO', 'WHI', 'WOR')
values <- c("IA0364", "IA1319", "IA1394", "IA5198", "IA4228", "IA0200")

new_times$station <- values[match(new_times$SiteID, index)]
flumeNA$station <- values[match(flumeNA$SiteID, index)]

In [ ]:
%%R
flume_sub <- new_times %>% 
  left_join(flumeNA, by = c("station","date")) %>%
  filter(!is.na(TSS_ppm)) %>%
  rename(SiteID = SiteID.x,
         Year = Year.x) %>%
  select(SiteID, Year, Treatment, date, flow_mm, rain_mm, TSS_ppm)

In [ ]:
%%R
write.csv(flume_sub,"./data/tidy/flume_sub.csv", row.names=F)

## Soil pad dataset

Loop through /data/weight and merge weight files into new dataframe. Rename columns to remove some of the poor naming structure and filter out 'NaN' values in response variable column.

In [ ]:
%%R
all_paths <- list.files(path = "./data/raw/weight/",
                        pattern = "*.csv",
                        full.names=TRUE)

In [ ]:
%%R
all_content <- all_paths %>%
lapply(read.table,
         header = TRUE,
         sep = ",",
         encoding = "UTF-8")

In [ ]:
%%R
all_filenames <- all_paths %>%
  basename() %>%
  as.list()

In [ ]:
%%R
all_lists <- mapply(c, all_content, all_filenames, SIMPLIFY = FALSE)

In [ ]:
%%R
all_result <- rbindlist(all_lists, fill = T) %>%
rename(PanID = Pan.ID,
       Cup_g = Cup..g.,
       Cup_soil = Cup...soil,
       filename = V1)

Modify columns in weight all_results dataframe.

In [ ]:
%%R
all_raw <- all_result %>%
mutate(filename = gsub("\\.[^.]*$", "", filename))

In [ ]:
%%R
split_into_multiple <- function(column, pattern = ", ", into_prefix){
  cols <- str_split_fixed(column, pattern, n = Inf)
  # Sub out the ""'s returned by filling the matrix to the right, with NAs which are useful
  cols[which(cols == "")] <- NA
  cols <- as.tibble(cols)
  # name the 'cols' tibble as 'into_prefix_1', 'into_prefix_2', ..., 'into_prefix_m' 
  # where m = # columns of 'cols'
  m <- dim(cols)[2]

  names(cols) <- paste(into_prefix, 1:m, sep = "_")
  return(cols)
}

In [ ]:
%%R
weight_raw <- all_raw %>% 
  bind_cols(split_into_multiple(.$filename, "_", "type")) %>%
  rename(Year = type_1,
         SiteID = type_2,
         Time = type_3) %>%
  mutate(weight = Cup_soil - Cup_g,
         trt_code = case_when(endsWith(Treatment, 'l') ~ "C",
                              endsWith(Treatment, 's') ~ "T"),
         PanID = str_pad(PanID, 2, pad="0"),
         PadID = paste(SiteID, PanID, trt_code, sep=""),
         Year = as.double(Year),
         Time = as.double(Time)) %>%
  filter(!is.na(weight)) %>%
  # selecting those that start with 'type_' will remove the original 'type' column
  select(Year, SiteID, Treatment, Position, Time, PadID, weight)

Merge weight measurements observed on the pads with experiment characteristics organized by site, year and time collected in order to add number of days within an observational period and amount of precipitation.

In [ ]:
%%R
site_year_time <- times %>%
mutate(EndDate = as.Date(EndDate,   format="%m/%d/%Y"),
       StartDate = as.Date(StartDate, format="%m/%d/%Y"),
       Days = as.numeric(EndDate - StartDate))

In [ ]:
%%R
weight <- weight_raw %>%
left_join(site_year_time, by = c("SiteID","Year","Time")) %>%
mutate(wpd = weight/Days) %>%
select(Year, SiteID, Treatment, Position, Time, PadID, weight, Mppt, Days, wpd)

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
write.csv(weight,"./data/tidy/weight.csv", row.names=F)

UsageError: Cell magic `%%R` not found.
